# **Soft Labels**

### ¿Qué son los Soft Labels?
Los **soft labels** (etiquetas suaves) son probabilidades asociadas a cada clase frente a etiquetas duras (hard labels). 
- En lugar de asignar una clase específica (por ejemplo, `0` o `1`), las soft labels representan la probabilidad de pertenencia de una muestra a cada clase.

Por ejemplo:
- Hard label: `[0]` (la muestra pertenece a la clase 0).
- Soft label: `[0.8, 0.2]` (la muestra tiene un 80% de probabilidad de pertenecer a la clase 0 y un 20% de pertenecer a la clase 1).

### ¿Por qué usar Soft Labels?
1. **Incorporación de incertidumbre**: Las soft labels permiten modelar la incertidumbre en las etiquetas.
2. **Mejora en el entrenamiento**: Algunos modelos pueden beneficiarse de soft labels para aprender patrones más robustos.
3. **Transferencia de conocimiento**: En aprendizaje por transferencia, las soft labels generadas por un modelo maestro pueden usarse para entrenar un modelo estudiante (técnica conocida como *knowledge distillation*).

## 2. Ejemplo Práctico con Scikit-Learn

En este ejemplo, generaremos datos sintéticos y usaremos soft labels para entrenar un modelo de regresión logística.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
df = pd.read_csv('../../data/titanic/train.csv')
df.columns

df.dropna(inplace=True)

In [ ]:
X = df.drop(['PassengerId', 'Survived', 'Pclass', 'Name',
            'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

y = df['Survived']
X, y

In [ ]:
lenc = LabelEncoder()
stadscl = StandardScaler()

X['Sex'] = lenc.fit_transform(X['Sex'])
X = stadscl.fit_transform(X)

print('X:', X[:5])

In [ ]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Clasificador clásico

In [ ]:
class_model = LogisticRegression(random_state=42)
class_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
y_pred = class_model.predict(X_test)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

### Crear Soft Labels
Generamos soft labels simulando probabilidades para cada clase. Usaremos un modelo maestro (en este caso, también una regresión logística) para generar estas probabilidades.


In [ ]:
# Entrenar un modelo maestro para generar soft labels
master_model = LogisticRegression(random_state=42)
master_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
soft_labels_train = master_model.predict_proba(X_train)

# Mostrar las primeras 5 soft labels
print("Soft Labels (primeras 5 filas):")
print(soft_labels_train[:5])

### Entrenar un Modelo Estudiante con Soft Labels
Usamos las soft labels generadas para entrenar un modelo lineal

In [ ]:
# Entrenar un modelo con soft labels
reg_model = LinearRegression()
# Usamos la probabilidad de la clase 1
reg_model.fit(X_train, soft_labels_train[:, 1])

# Predecir en el conjunto de prueba
y_pred = reg_model.predict(X_test)
print(y_pred)
y_pred_proba = (y_pred > 0.5).astype(int)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")

## 3. Resultados y Conclusión

### Resultados
- El modelo estudiante se entrena con soft labels generadas por el modelo maestro.
- Las soft labels permiten al modelo estudiante aprender patrones más suaves y generalizables.

### Conclusión
- Los soft labels son útiles para incorporar incertidumbre en el proceso de entrenamiento.
- Técnicas como *knowledge distillation* utilizan soft labels para transferir conocimiento de un modelo grande a uno más pequeño.

## Ejercicio:
1. Entrenar un modelo de forecasting para predecir predecir si el consumo de combustible subirá o bajar a 12 meses vista usando sof labels.

- Usa el dataset 'fuel_consumption'.
- Usa frecuancia mensual.

ref. https://github.com/skforecast/skforecast-datasets

In [41]:
import pandas as pd
from skforecast.datasets import fetch_dataset
from tabulate import tabulate

# Cargar el dataset fuel_consumption
data = fetch_dataset("fuel_consumption")

# Crear la variable objetivo: 1 si el consumo sube, 0 si baja
data['target'] = (data['GLPs'].diff() > 0).astype(int)

# Eliminar valores faltantes generados por la diferenciación
data = data.dropna()

# Mostrar los primeros registros
print(tabulate(data.head(20),   headers='keys', tablefmt='psql'))


fuel_consumption
----------------
Monthly fuel consumption in Spain from 1969-01-01 to 2022-08-01.
Obtained from Corporación de Reservas Estratégicas de Productos Petrolíferos and
Corporación de Derecho Público tutelada por el Ministerio para la Transición
Ecológica y el Reto Demográfico. https://www.cores.es/es/estadisticas
Shape of the dataset: (644, 5)
+---------------------+----------+-------------+--------------+------------+------------------+----------+
| Fecha               |     GLPs |   Gasolinas |   Querosenos |   Gasoleos |        Fueloleos |   target |
|---------------------+----------+-------------+--------------+------------+------------------+----------|
| 1969-01-01 00:00:00 | 133615   |      166875 |       123258 |     401185 | 912583           |        0 |
| 1969-02-01 00:00:00 | 126748   |      155467 |       114683 |     385360 | 851878           |        0 |
| 1969-03-01 00:00:00 | 107796   |      184984 |       109970 |     418956 | 873884           |        0 |


In [42]:
X= data.drop(['target','GLPs'], axis=1)
y= data['target']

X, y

(              Gasolinas   Querosenos      Gasoleos    Fueloleos
 Fecha                                                          
 1969-01-01  166875.2129  123257.8090  4.011853e+05  912583.4202
 1969-02-01  155466.8105  114682.5767  3.853600e+05  851877.8115
 1969-03-01  184983.6699  109970.0796  4.189556e+05  873884.2933
 1969-04-01  202319.8164  108797.9255  4.389755e+05  755490.1170
 1969-05-01  206259.1523  103554.0784  4.674519e+05  729963.6009
 ...                 ...          ...           ...          ...
 2022-04-01  471601.9400  494966.3200  2.638030e+06  636930.4200
 2022-05-01  478873.4100  530034.0100  2.673797e+06  708289.2600
 2022-06-01  501447.1400  540594.9700  2.648087e+06  667221.4500
 2022-07-01  534584.6800  607854.6800  2.613591e+06  691371.6000
 2022-08-01  565761.9600  596051.0800  2.572012e+06  701384.4100
 
 [644 rows x 4 columns],
 Fecha
 1969-01-01    0
 1969-02-01    0
 1969-03-01    0
 1969-04-01    0
 1969-05-01    0
              ..
 2022-04-01    0
 2

In [43]:
# Dividir en entrenamiento y prueba
X_train, X_test = X[:-12], X[-12:]
y_train, y_test = y[:-12], y[-12:]

X_train.tail(), y_test.head()

(            Gasolinas  Querosenos    Gasoleos  Fueloleos
 Fecha                                                   
 2021-04-01  377438.65   140343.00  2448229.67  505302.60
 2021-05-01  432848.68   176393.81  2520854.77  484193.72
 2021-06-01  483500.45   248326.60  2628985.76  508827.10
 2021-07-01  540217.21   394659.97  2693994.26  517741.86
 2021-08-01  540653.12   426644.89  2525759.39  578165.99,
 Fecha
 2021-09-01    1
 2021-10-01    1
 2021-11-01    1
 2021-12-01    1
 2022-01-01    0
 Freq: MS, Name: target, dtype: int64)

In [44]:
# Entrenar un modelo maestro para generar soft labels
master_model = LogisticRegression(random_state=42)
master_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
soft_labels_train = pd.DataFrame( master_model.predict_proba(X_train), index=X_train.index, columns=['P0', 'P1'])

# Mostrar las primeras 5 soft labels
print("Soft Labels (primeras 5 filas):")
print(soft_labels_train[5:])

Soft Labels (primeras 5 filas):
                  P0        P1
Fecha                         
1969-06-01  0.506419  0.493581
1969-07-01  0.511182  0.488818
1969-08-01  0.510120  0.489880
1969-09-01  0.499918  0.500082
1969-10-01  0.493242  0.506758
...              ...       ...
2021-04-01  0.279623  0.720377
2021-05-01  0.307566  0.692434
2021-06-01  0.357848  0.642152
2021-07-01  0.489086  0.510914
2021-08-01  0.540876  0.459124

[627 rows x 2 columns]


In [45]:
# Separación datos train-test

steps = 12
datos_train = soft_labels_train
# datos_test  = soft_labels_train[-steps:]

print(f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
# print(f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")

Fechas train : 1969-01-01 00:00:00 --- 2021-08-01 00:00:00  (n=632)


In [49]:
from skforecast.recursive import ForecasterRecursive

# Crear el forecaster recursivo con un modelo de regresión lineal
forecaster = ForecasterRecursive(
    regressor=LinearRegression(),
    lags=12  # Predecir 12 pasos hacia el futuro
)


forecaster.fit(y=datos_train['P1'])
forecaster

=================== 
ForecasterRecursive 
=================== 
Regressor: LinearRegression 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
Window features: None 
Window size: 12 
Exogenous included: False 
Exogenous names: None 
Transformer for y: None 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1969-01-01 00:00:00'), Timestamp('2021-08-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2025-04-10 10:00:33 
Last fit date: 2025-04-10 10:00:33 
Skforecast version: 0.15.1 
Python version: 3.10.11 
Forecaster id: None

In [50]:
# Predicciones

y_pred = forecaster.predict(steps=steps)
y_pred

2021-09-01    0.498811
2021-10-01    0.540225
2021-11-01    0.596730
2021-12-01    0.642612
2022-01-01    0.694055
2022-02-01    0.741014
2022-03-01    0.730721
2022-04-01    0.698947
2022-05-01    0.667746
2022-06-01    0.610448
2022-07-01    0.551152
2022-08-01    0.514837
Freq: MS, Name: pred, dtype: float64

In [51]:
# Predecir en el conjunto de prueba
y_pred_proba = (y_pred > 0.5).astype(int)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.4167


In [ ]:
# Convertir la columna 'Fecha' a índice temporal
type(data.index)

In [ ]:
X= data.drop(['target'], axis=1)
y= data['target']

In [ ]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenar un modelo maestro para generar soft labels
master_model = LogisticRegression(random_state=42)
master_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
soft_labels_train = pd.DataFrame( master_model.predict_proba(X_train), index=X_train.index, columns=['P0', 'P1'])

# Mostrar las primeras 5 soft labels
print("Soft Labels (primeras 5 filas):")
print(soft_labels_train[:5])

In [ ]:
# Separación datos train-test

steps = 12
datos_train = soft_labels_train[:-steps]
datos_test  = soft_labels_train[-steps:]

print(f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")

In [ ]:
from skforecast.recursive import ForecasterRecursive

# Crear el forecaster recursivo con un modelo de regresión lineal
forecaster = ForecasterRecursive(
    regressor=LinearRegression(),
    lags=12  # Predecir 12 pasos hacia el futuro
)


forecaster.fit(y=datos_train['P0'])
forecaster

In [ ]:
# Predicciones

y_pred = forecaster.predict(steps=steps)
y_pred

In [ ]:
# Predecir en el conjunto de prueba
y_pred_proba = (y_pred > 0.5).astype(int)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test[:12], y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")